In [211]:
#http://geologyandpython.com/toa-landsat-8.html
#https://nbviewer.jupyter.org/github/mapbox/rio-cloudmask/blob/master/examples/Landsat8_Examples.ipynb

In [7]:
from glob import glob
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from l8qa.qa import write_cloud_mask
from rio_toa import reflectance
from rio_toa import sun_utils, toa_utils
from rio_toa.toa_utils import rescale, _parse_bands_from_filename
from rio_toa.reflectance import calculate_landsat_reflectance
from rio_toa.brightness_temp import calculate_landsat_brightness_temperature
from rio_color.operations import parse_operations
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
%matplotlib inline


In [2]:
##create cloud mask

In [3]:
for folder in landsat_folders_path:
    print(folder)
    folder_name = os.path.split(src_folder[:-1])[-1]
    dst_folder = os.path.join(folder, folder_name+'_TOA')
    os.makedirs(dst_folder, exist_ok=True)
    search_criteria = "*.TIF"
    glob_creteria= os.path.join(folder, search_criteria)
    src_mtl = glob(os.path.join(src_folder, '*MTL*'))[0]
    src_qa = glob(os.path.join(src_folder, '*QA*'))[0]
    with rasterio.open(src_qa) as qa_raster:
        # Update the raster profile to use 0 as 'nodata'
        profile = qa_raster.profile
        profile.update(nodata=0)
        #Call `write_cloud_mask` to create a mask where the QA points to clouds.
        write_cloud_mask(qa_raster.read(1), profile, os.path.join(dst_folder, folder_name+'_cloudmask.TIF'))
    print('----------------------')



NameError: name 'landsat_folders_path' is not defined

In [68]:
#calculate TOA

In [8]:
def read_toa(fname,dst_path, mtl):
    creation_options = {'nodata': 0,'compress': 'deflate','predict': 2}
    try:
        bandnum = _parse_bands_from_filename([fname], '.*/LC08.*\_B{b}.TIF')
        print(bandnum)
    
    
    

        if bandnum[0] in (10, 11):
            dir_path=os.path.dirname(dst_path)
            folder_name = os.path.split(dst_path)[-1]
            folder_name=folder_name.replace('TOA','BTMP')
            dst_path=dir_path+'/'+folder_name
            #print(dst_path)
            calculate_landsat_brightness_temperature(
                fname, mtl, dst_path, 'C', {}, 10, 'float32', 8)
            
        else:
            calculate_landsat_reflectance(
                [fname], mtl, dst_path,rescale_factor = 55000,
                creation_options= creation_options, 
                bands=bandnum, dst_dtype = 'uint16', processes = 4, 
                pixel_sunangle=True)
            '''
            with rasterio.open(dst_path) as src:
            arr = src.read(1)
            return arr
            ''' 
    except ValueError:
            pass

In [9]:
def create_toa(landsat_folders_path):
        
    if type(landsat_folders_path)==type(list()) :
        
        for folder in landsat_folders_path:
            #print(folder)
            folder_name = os.path.split(folder[:-1])[-1]
            dst_folder = os.path.join(folder, folder_name+'_TOA')
            os.makedirs(dst_folder, exist_ok=True)
            search_criteria = "*.TIF"
            glob_creteria = os.path.join(folder, search_criteria)       
            src_mtl = glob(os.path.join(folder, '*MTL*'))[0]
            landsat_bands = glob(glob_creteria)
            landsat_bands=sorted(landsat_bands)
            #print(landsat_bands)
            
            for i in landsat_bands:
                if i[-7:-4]=='BQA':
                    continue
                else:
    
                    img_name='TOA_'+os.path.basename(i)
                    dst_path = os.path.join(dst_folder,img_name)
                    #print(i)
                    #print(dst_path)
                    #print(src_mtl)
                    read_toa(i,dst_path,src_mtl)
                    
                    print(dst_path)               
                    
            print('------------')
    else:
            folder_name = os.path.split(landsat_folders_path[:-1])[-1]
            dst_folder = os.path.join(landsat_folders_path, folder_name+'_TOA')
            print(dst_folder)
            print('--------folder----')
            os.makedirs(dst_folder, exist_ok=True)
            search_criteria = "*.TIF"
            glob_creteria = os.path.join(landsat_folders_path, search_criteria)       
            src_mtl = glob(os.path.join(landsat_folders_path, '*MTL*'))[0]
            landsat_bands = glob(glob_creteria)
            landsat_bands=sorted(landsat_bands)
            for i in landsat_bands:
                if i[-7:-4]=='BQA':
                    continue
                else:
                    img_name='TOA_'+os.path.basename(i)
                    dst_path = os.path.join(dst_folder,img_name)
                    read_toa(i,dst_path,src_mtl)
                    print('------Calculate finished------')
                    print(dst_path)               
                    
            print('------------')


In [10]:
SRC_LANDSAT_FOLDER = '../mardin/'
#Find Landsat images' folder 
landsat_folders_path = glob(os.path.join(SRC_LANDSAT_FOLDER, 'L*'))
landsat_folders_path

['../mardin/LC08_L1TP_172034_20180704_20180717_01_T1',
 '../mardin/LC08_L1TP_172035_20180704_20180717_01_T1',
 '../mardin/LC08_L1TP_173034_20180711_20180717_01_T1']

In [11]:
landsat_folders_path=sorted(landsat_folders_path)
landsat_folders_path

['../mardin/LC08_L1TP_172034_20180704_20180717_01_T1',
 '../mardin/LC08_L1TP_172035_20180704_20180717_01_T1',
 '../mardin/LC08_L1TP_173034_20180711_20180717_01_T1']

In [12]:
create_toa(landsat_folders_path)

[2]
../mardin/LC08_L1TP_172034_20180704_20180717_01_T1/LC08_L1TP_172034_20180704_20180717_01_T_TOA/TOA_LC08_L1TP_172034_20180704_20180717_01_T1_B2.TIF
[3]
../mardin/LC08_L1TP_172034_20180704_20180717_01_T1/LC08_L1TP_172034_20180704_20180717_01_T_TOA/TOA_LC08_L1TP_172034_20180704_20180717_01_T1_B3.TIF
[4]
../mardin/LC08_L1TP_172034_20180704_20180717_01_T1/LC08_L1TP_172034_20180704_20180717_01_T_TOA/TOA_LC08_L1TP_172034_20180704_20180717_01_T1_B4.TIF
[5]
../mardin/LC08_L1TP_172034_20180704_20180717_01_T1/LC08_L1TP_172034_20180704_20180717_01_T_TOA/TOA_LC08_L1TP_172034_20180704_20180717_01_T1_B5.TIF
------------
[2]
../mardin/LC08_L1TP_172035_20180704_20180717_01_T1/LC08_L1TP_172035_20180704_20180717_01_T_TOA/TOA_LC08_L1TP_172035_20180704_20180717_01_T1_B2.TIF
[3]
../mardin/LC08_L1TP_172035_20180704_20180717_01_T1/LC08_L1TP_172035_20180704_20180717_01_T_TOA/TOA_LC08_L1TP_172035_20180704_20180717_01_T1_B3.TIF
[4]
../mardin/LC08_L1TP_172035_20180704_20180717_01_T1/LC08_L1TP_172035_20180704_